In [ ]:
# !pip install transformers accelerate jsonformer pymupdf

# Get pdf

In [ ]:
from pathlib import Path
ROOT_DIR = Path().resolve()

pdf_path = ROOT_DIR / "ISS.pdf"

# Extract text

In [ ]:
import fitz
import json

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = [page.get_text("text") for page in doc]
    return text

from pathlib import Path
ROOT_DIR = Path().resolve().parent

extracted_text = extract_text_from_pdf(pdf_path)
extracted_text

['Invoice To:\nDirect Produce Supplies LTD\n57-63 Church Street\nWimbledon\nSW19 5DQ\nInvoice Date: 22/11/2023\nInvoice No:22806\nQuantity\nRate (£)\nNet Value (£)\n178,082\n0.090\n10,641.96\n105,084\n0.090\n2,856.15\n36,065\n0.090\n1,762.74\nNet Total (£):\n15,260.85\nVAT (£):\n3,052.17\nBank Details: HSBC UK Bank Plc\nPayment Terms: 28 days from invoice date\nGross Total (£):\n18,313.02\nSort Code: 40-03-21\nAccount No: 2236 6789\nContact\n T 01795 523 310\n E accounts@issproduce.co.uk\nW www.issproduce.co.uk\nINTEGRATED SERVICE SOLUTIONS\nIntegrated Service Solutions Limited, C/O Fowler Welch, London Road, Teynham, Sittingbourne, ME9 9PR\nService Description\nISS SP - Picking: 12/11/2023 - 18/11/2023 Teynham\nISS SP - Picking: 12/11/2023 - 18/11/2023 Linton\nISS SP - Picking: 12/11/2023 - 18/11/2023 Sittingbourne\nVAT Registration No: 184 6959 49            Company Registration No: 08332191\n']

# Using very small model due to collab limitations

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading model and tokenizer...")
model_name = "databricks/dolly-v2-3b"
model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_cache=True)
print("Loaded model and tokenizer")

Loading model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Loaded model and tokenizer


In [ ]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

invoice_schema = {
    "type": "object",
    "properties": {
        "invoice_number": {"type": "string"},
        "invoice_date": {"type": "string"},
        "line_items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "product_code": {"type": "string"},
                    "description": {"type": "string"},
                    "quantity": {"type": "string"},
                    "price_per_unit": {"type": "string"},
                    "vat_percent": {"type": "string"},
                    "total_price": {"type": "string"}
                }
            }
        },
        },
    }


builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=invoice_schema,
    prompt=f"extract the list of invoice items {extracted_text}",
)

print("Generating...")
output = builder()

highlight_values(output)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating...
{
  invoice_number: "22806",
  invoice_date: "22/11/2023",
  line_items: [
    {
      product_code: "178-082",
      description: "Direct Produce Supplies LTD",
      quantity: "178,082",
      price_per_unit: "0.090",
      vat_percent: "10.641",
      total_price: "105,084.00"
    }
  ]
}


In [ ]:
output

{'invoice_number': '22806',
 'invoice_date': '22/11/2023',
 'line_items': [{'product_code': '178-082',
   'description': 'Direct Produce Supplies LTD',
   'quantity': '178,082',
   'price_per_unit': '0.090',
   'vat_percent': '10.641',
   'total_price': '105,084.00'}]}